In [4]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Oracle 생성
def create_simon_oracle(secret_string):
    n = len(secret_string)
    oracle = QuantumCircuit(2*n, name='Simon_oracle')

    for i in range(n):
        oracle.cs(i, n+1)

    for i, bit in enumerate(secret_string):
        if bit == '1':
            oracle.cx(0, n+1)

    return oracle

# 회로 생성
def simon_circuit(secret_string):
    n = len(secret_string)

    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'result')

    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    qc.h(input_qubits)
    qc.barrier()

    oracle = create_simon_oracle(secret_string)
    qc.append(oracle, input_qubits[:] +output_qubits[:])

    qc.h(input_qubits)
    qc.barrier()

    qc.measure(input_qubits, classical_bits)

    return qc

# 실제 양자컴
service = QiskitRuntimeService(
    channel = 'ibm_quantum_platform',
    token = '')

backend = service.least_busy(simulator = False, operational=True, min_num_qubits =6)
print(f"백엔드 : {backend.name}")

#회로 실행
SECRET = "110"
qc = simon_circuit(SECRET)

pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circuit = pm.run(qc)

sampler = Sampler(backend)
job = sampler.run([isa_circuit], shots =1024)
print(f"Job ID: {job.job_id()}")

result = job.result()
counts_raw = result[0].data.result.get_counts()
counts = {k[::-1]: v for k, v in counts_raw.items()}

#결과
for bitstring, count in sorted(counts.items(), key=lambda x : x[1], reverse=True)[:10]:
    print(f"|{bitstring}>: {count}회")

qiskit_runtime_service.__init__:WARNING:2025-12-21 22:22:53,565: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: dotnanimoe. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-12-21 22:22:53,954: Loading instance: dotnanimoe, plan: open
qiskit_runtime_service.backends:WARNING:2025-12-21 22:22:56,509: Using instance: dotnanimoe, plan: open


백엔드 : ibm_torino
Job ID: d53v9chsmlfc739f9740
|000>: 977회
|010>: 13회
|100>: 13회
|001>: 10회
|101>: 4회
|110>: 3회
|011>: 3회
|111>: 1회
